# Data exploration
In this notebook we read in the data and visualize it to see it is clean,etc.

At the end the data gets written away to train the model.

In early versions this will all be done in the same notebook.

#### Import and checks

In [ ]:
# Import libraries
import torch
from torchvision import datasets, transforms
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from src import model_def as md
from src import data_processing as dp

In [ ]:
# check if Pytorch CUDA is installed correctly

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("CUDA Device Name:", torch.cuda.get_device_name(0))

print("CUDA Version:", torch.version.cuda)
print("cuDNN Version:", torch.backends.cudnn.version())

#### Pre-sorting

Sort data accoridng to provided label of the given classification of the dataset (data_entry_2017.csv)

Multiple findings/diagnoses means that a copy gets put into each finding folder. This to get as much samples as possible in each outcome for training purposes.

Both sort test and train images. TODO: do this without two seperate loops/make it a function.

In [ ]:
csv_path = os.path.join("..", "data", "classification", "Data_Entry_2017_v2020.csv")
image_folder = os.path.join("..", "data", "images_train")
output_folder = os.path.join("..", "data", "images_sorted_train")

dp.organize_images(csv_path, image_folder, output_folder)

image_folder = os.path.join("..", "data", "images_test")
output_folder = os.path.join("..", "data", "images_sorted_test")

dp.organize_images(csv_path, image_folder, output_folder)

#### Data loading

Import a testset to be used in training

We normalize the input since we work in grayscale. No size transformations are done since the dataset is already in 1024X1024.

In [ ]:
train_image_folder = os.path.join("..", "data", "images_sorted_train")
test_image_folder = os.path.join("..", "data", "images_sorted_test")
# print(train_image_folder)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.Grayscale(num_output_channels=1), 
])

dataset_xrays_train = datasets.ImageFolder(root=train_image_folder, transform=transform)
dataset_xrays_test = datasets.ImageFolder(root=test_image_folder, transform=transform)

print(dataset_xrays_train)

#### Model setup
Below the actual model is trained

In [ ]:
# num_class = 0
# for subobject in os.listdir(output_folder):
#     subobject_path = os.path.join(output_folder, subobject)

#     if os.path.isdir(subobject_path):
#         num_class+=1

model, data_loader, criterion, optimizer, device = md.setup_model_and_training(dataset_xrays_train)
data_loader_test = md.setup_dataloader(dataset_xrays_test)
model = md.train_model(model, data_loader, criterion, optimizer, device, num_epochs=10)


#### Evaluate the model
First eval against the train set, then evala gainst test set

In [ ]:

metrics = md.evaluate_model(model, data_loader, device, class_names=dataset_xrays_train.classes)
# md.visualize_predictions(model, data_loader, device, num_examples=5, class_names=dataset_xrays_test.classes)
misclass_df = md.analyze_misclassifications(model, data_loader, device, class_names=dataset_xrays_train.classes)
hard_examples = md.find_hard_examples(model, data_loader, device, n=5)

Eval against test set

In [ ]:

metrics = md.evaluate_model(model, data_loader_test, device, class_names=dataset_xrays_test.classes)
# md.visualize_predictions(model, data_loader_test, device, num_examples=5, class_names=dataset_xrays_test.classes)
misclass_df = md.analyze_misclassifications(model, data_loader_test, device, class_names=dataset_xrays_test.classes)
hard_examples = md.find_hard_examples(model, data_loader_test, device, n=5)